In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

# %pdb on

# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]
# NPBのMGの初期変数
mg_size :list[int] = [32, 64, 128, 256, 512]
mg_nit: list[int] = [4, 10, 20, 35, 50]

train_mg_size :list[int] = mg_size[:-1]
train_mg_nit :list[int] = mg_nit[:-1]

test_mg_size :list[int] = mg_size[-1:]
test_mg_nit :list[int] = mg_nit[-1:]


# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

/workspace


DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


In [2]:
# 学習用のデータ
trainDF_cg :pd.DataFrame = return_rawDF_cg(
    list_process=train_npb_process,
    list_na=train_cg_na,
    list_nonzer=train_cg_nonzer,
    list_niter=train_cg_niter,
    list_shift=train_cg_shift,
    csvDir=csvDirPath
)

# 予測対象用のデータ
testDF_cg :pd.DataFrame = return_rawDF_cg(
    list_process=test_npb_process,
    list_na=test_cg_na,
    list_nonzer=test_cg_nonzer,
    list_niter=test_cg_niter,
    list_shift=test_cg_shift,
    csvDir=csvDirPath
)

# コール回数モデル構築
# 1コール当たりのExclusiveモデル構築


/workspace/lib/lab_lib.py:8785: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/workspace/lib/lab_lib.py:8785: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [ ]:
def return_rawDF_mg(
    list_process: list[int],
    list_nit: list[int],
    list_size: list[int],
    csvDir: str,
) -> pd.DataFrame:

    """return_rawDF_mg()
    
    ベンチマークプログラムMGの手動で変更した初期変数におけるプロファイルを取得する関数

    Args:
        list_process(list[int]):プロセス数のリスト
        list_nit(list[int]):初期変数nitのリスト
        list_size(list[int]):初期変数sizeのリスト
        csvDir(str):CSVファイルを格納したディレクトリのパスを表す文字列

    Returns:
        pd.DataFrame
    
    """

    list_before_concat_DF: list[pd.DataFrame] = []

    for elem_process in list_process:
        for elem_nit in list_nit:
            for elem_size in list_size:
                filePath: str = f"{csvDir}mg_problem_size{elem_size}_nit{elem_nit}_process{elem_process}.csv"
                if os.path.isfile(filePath):
                    try:
                        DF_read_raw: pd.DataFrame = pd.read_csv(filePath)
                        DF_read_raw["process"] = elem_process
                        DF_read_raw["size"] = elem_size
                        DF_read_raw["nit"] = elem_nit
                        list_before_concat_DF.append(DF_read_raw)
                    except:
                        warnings.warn(f"{filePath} is empty.")
                else:
                    warnings.warn(f"{filePath} doesn't exist")
    return pd.concat(objs=list_before_concat_DF, axis=0)